In [126]:
from scipy.io import arff
import pandas as pd
import numpy as np
import operator
import math

In [127]:
Dtrain =arff.loadarff('trainProdSelection.arff')
Dtest =arff.loadarff('testProdSelection.arff')

In [128]:
Dtrain = pd.DataFrame(Dtrain[0])
Dtest = pd.DataFrame(Dtest[0])

In [129]:
def normalize (x): 
    return ( (x-np.min(x))/ (max(x) - min(x)))

In [130]:
df1 = pd.get_dummies(Dtrain['Type'],drop_first=True)

In [131]:
df2 = pd.get_dummies(Dtrain['LifeStyle'],drop_first=True)

In [132]:
Dtrain = pd.concat([Dtrain,df1,df2],axis=1)

In [133]:
df3 = pd.get_dummies(Dtest['Type'],drop_first=True)
df4 = pd.get_dummies(Dtest['LifeStyle'],drop_first=True)

In [134]:
Dtest = pd.concat([Dtest,df3,df4],axis=1)

In [135]:
Dtrain.drop(['Type','LifeStyle'],axis=1,inplace=True)

In [136]:
Dtest.drop(['Type','LifeStyle'],axis=1,inplace=True)

In [137]:
Dtrain.head()

,Vacation,eCredit,salary,property,label,b'engineer',b'librarian',b'professor',b'student',b'spend<saving',b'spend>>saving',b'spend>saving'
0,6.0,40.0,13.62,3.2804,b'C1',0,0,0,1,0,0,1
1,11.0,21.0,15.32,2.0232,b'C1',0,0,0,1,0,0,1
2,7.0,64.0,16.55,3.1202,b'C1',0,0,0,1,0,0,1
3,3.0,47.0,15.71,3.4022,b'C1',0,0,0,1,0,0,1
4,15.0,10.0,16.96,2.2825,b'C1',0,0,0,1,0,0,1


In [138]:
Dtest.head()

,Vacation,eCredit,salary,property,label,b'engineer',b'librarian',b'professor',b'student',b'spend<saving',b'spend>>saving',b'spend>saving'
0,12.0,19.0,14.7900,3.7697,b'C1',0,0,0,1,1,0,0
1,29.0,10.0,16.1900,2.4839,b'C1',0,0,0,1,0,1,0
2,28.0,60.0,15.4600,1.1885,b'C1',0,0,0,1,0,0,0
3,15.0,41.0,21.2600,1.4379,b'C1',1,0,0,0,0,0,1
4,2.0,9.0,19.7207,0.6913,b'C1',0,1,0,0,1,0,0


In [139]:
train=Dtrain[['Vacation','eCredit','salary','property',b'engineer',b'librarian',b'professor',b'student',b'spend<saving',
 b'spend>>saving',b'spend>saving']].apply(normalize)

In [140]:
test =Dtest[['Vacation','eCredit','salary','property',b'engineer',b'librarian',b'professor',b'student',b'spend<saving',
 b'spend>>saving',b'spend>saving']].apply(normalize)

In [141]:
train = pd.concat([train,Dtrain['label']],axis=1)

In [142]:
test = pd.concat([test,Dtest['label']],axis=1)

In [143]:
train.head()

,Vacation,eCredit,salary,property,b'engineer',b'librarian',b'professor',b'student',b'spend<saving',b'spend>>saving',b'spend>saving',label
0,0.079365,0.107558,0.219960,0.183167,0.0,0.0,0.0,1.0,0.0,0.0,1.0,b'C1'
1,0.158730,0.052326,0.293102,0.112797,0.0,0.0,0.0,1.0,0.0,0.0,1.0,b'C1'
2,0.095238,0.177326,0.346023,0.174200,0.0,0.0,0.0,1.0,0.0,0.0,1.0,b'C1'
3,0.031746,0.127907,0.309882,0.189984,0.0,0.0,0.0,1.0,0.0,0.0,1.0,b'C1'
4,0.222222,0.020349,0.363663,0.127311,0.0,0.0,0.0,1.0,0.0,0.0,1.0,b'C1'


In [144]:
test.head()

,Vacation,eCredit,salary,property,b'engineer',b'librarian',b'professor',b'student',b'spend<saving',b'spend>>saving',b'spend>saving',label
0,0.20,0.058824,0.104637,0.398926,0.0,0.0,0.0,1.0,1.0,0.0,0.0,b'C1'
1,0.54,0.021008,0.175059,0.243041,0.0,0.0,0.0,1.0,0.0,1.0,0.0,b'C1'
2,0.52,0.231092,0.138339,0.085992,0.0,0.0,0.0,1.0,0.0,0.0,0.0,b'C1'
3,0.26,0.151261,0.430086,0.116229,1.0,0.0,0.0,0.0,0.0,0.0,1.0,b'C1'
4,0.00,0.016807,0.352657,0.025714,0.0,1.0,0.0,0.0,1.0,0.0,0.0,b'C1'


### Finding distance between two given points by euclidian metric

In [145]:
def euclideanMetric(x1, x2, length):
    distance = 0
    for x in range(0,length):
        distance += pow((x1[x] - x2[x]), 2)
    return math.sqrt(distance)

### Finding the nearest neighbour among K nearest neighbours using majority vote

In [146]:
def KthNeighbor(neighbors):
    Votes = {}
    for x in range(len(neighbors)):
        r = neighbors[x][-1]
        if r in Votes:
            Votes[r] += 1
        else:
            Votes[r] = 1
    sortedVotes = sorted(Votes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

### Finding the accuracy of the algorithm

In [147]:
def AccuracyPercentage(test, results):
    c = 0
    for x in range(len(test)):
        if test[x][-1] == results[x]:
            c += 1
    return (c/float(len(test))) * 100.0

### Finding K nearest neighbours for all test points 

In [148]:
def NearestNeighbors(training, test, k):
    distances = []
    length = len(test)-1
    for x in range(len(training)):
        dist = euclideanMetric(test, training[x], length)
        distances.append((training[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

### Main Function

In [149]:
k =3
predictions=[]
for x in range(len(test)):
    n = NearestNeighbors(train.values,test.values[x],k)
    r = KthNeighbor(n)
    predictions.append(r)
accuracy = AccuracyPercentage(test.values, predictions)
print('Accuracy')
print(' '+repr(accuracy) + '%')

Accuracy
 28.57142857142857%
